In [3]:
import torch
import pandas as pd
import Utility
import SQLSentence
from torch.utils.data import Dataset, DataLoader
import tqdm

%load_ext autoreload
%autoreload 1
%aimport SQLSentence
%aimport Utility

In [4]:
# Build Basic Data

connect = Utility.connect_to_database()
data = Utility.GetAllData(connect)
list_stock_type = (SQLSentence.QuerySQL('stock.stock_code_type', connect, ['distinct stock_type_id']))['stock_type_id'].tolist()
df_code_name = SQLSentence.QuerySQL('stock.stock_code', connect)
dict_code_name = dict(zip(df_code_name.code, df_code_name.name))
dict_type_name = ((SQLSentence.QuerySQL('stock.stock_type', connect)).drop(['id'], axis=1)).to_dict()['name']

del df_code_name

In [5]:
# Constants
WINDOW_LENGTH_TRAIN = 20
WINDOW_LENGTH_TARGET = 1

In [8]:
class StockDataset(Dataset):
    def __init__(self, data, window_length_train, window_length_target):
        self.data = data
        self.window_length_train = window_length_train
        self.window_length_target = window_length_target

        self.data = Utility.ProcessData(self.data)

        #處理資料、新增指標

    def __getitem__(self, index):
        # index 要加 window size
        return torch.tensor(self.data[index:index+self.window_length_train], dtype=torch.float64).flatten()

    def __len__(self):
        return len(self.data) - self.window_length_train - self.window_length_target

In [15]:
for type_id in tqdm.tqdm(list_stock_type):
    list_unique_code_from_data = (SQLSentence.GetCodeByTypeId(type_id, connect))['code'].tolist()
    for code in list_unique_code_from_data:
        dataset = StockDataset(data[data['code'] == code], WINDOW_LENGTH_TRAIN, WINDOW_LENGTH_TARGET)

100%|██████████| 34/34 [01:09<00:00,  2.05s/it]
